# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Shi  ->  J. Shi  |  ['J. Shi']
J. Li  ->  J. Li  |  ['J. Li']
W. Laun  ->  W. Laun  |  ['W. Laun']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
T. Henning  ->  T. Henning  |  ['T. Henning']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
T. Henning  ->  T. Henning  |  ['T. Henning']
J. Li  ->  J. Li  |  ['J. Li']
S. Ghosh  ->  S. Ghosh  |  ['S. Ghosh']
Arxiv has 68 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2407.19152


extracting tarball to tmp_2407.19152...

 done.
Retrieving document from  https://arxiv.org/e-print/2407.19388


extracting tarball to tmp_2407.19388...

...

Found 93 bibliographic references in tmp_2407.20066/aa.bbl.
Retrieving document from  https://arxiv.org/e-print/2407.20118


extracting tarball to tmp_2407.20118... done.
Retrieving document from  https://arxiv.org/e-print/2407.20220


extracting tarball to tmp_2407.20220...

 done.


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2779/1211882699.py:51: LatexWarning: 2407.20220 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.19919-b31b1b.svg)](https://arxiv.org/abs/2407.19919) | **Euclid preparation. Exploring the properties of proto-clusters in the Simulated Euclid Wide Survey**  |
|| E. Collaboration, et al. -- incl., <mark>E. Bañados</mark>, <mark>K. Jahnke</mark> |
|*Appeared on*| *2024-07-30*|
|*Comments*| *Submitted to Astronomy and Astrophysics, 24 pages, 28 figures*|
|**Abstract**|            Galaxy proto-clusters are receiving an increased interest since most of the processes shaping the structure of clusters of galaxies and their galaxy population are happening at early stages of their formation. The Euclid Survey will provide a unique opportunity to discover a large number of proto-clusters over a large fraction of the sky (14 500 square degrees). In this paper, we explore the expected observational properties of proto-clusters in the Euclid Wide Survey by means of theoretical models and simulations. We provide an overview of the predicted proto-cluster extent, galaxy density profiles, mass-richness relations, abundance, and sky-filling as a function of redshift. Useful analytical approximations for the functions of these properties are provided. The focus is on the redshift range z= 1.5 to 4. We discuss in particular the density contrast with which proto-clusters can be observed against the background in the galaxy distribution if photometric galaxy redshifts are used as supplied by the ESA Euclid mission together with the ground-based photometric surveys. We show that the obtainable detection significance is sufficient to find large numbers of interesting proto-cluster candidates. For quantitative studies, additional spectroscopic follow-up is required to confirm the proto-clusters and establish their richness.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.19969-b31b1b.svg)](https://arxiv.org/abs/2407.19969) | **The CARMENES search for exoplanets around M dwarfs: Magnesium and silicon abundances of K7-M5.5 stars**  |
|| H. M. Tabernero, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2024-07-30*|
|*Comments*| *11 Pages, 5 figures. Accepted for publication in A&A*|
|**Abstract**|            We present the abundances of magnesium (Mg) and silicon (Si) for 314 dwarf stars with spectral types in the interval K7.0-M5.5 (Teff range ~4200-3050 K) observed with the CARMENES high-resolution spectrograph at the 3.5 m telescope at the Calar Alto Observatory. Our analysis employs the BT-Settl model atmospheres, the radiative transfer code Turbospectrum, and a state-of-the-art selection of atomic and molecular data. These Mg and Si abundances are critical for understanding both the chemical evolution and assembly of the Milky Way and the formation and composition of rocky planets. Our chemical abundances show a line-to-line scatter at the level of 0.1 dex for all studied spectral types. The typical error bar of our chemical abundance measurements is +- 0.11 dex (Mg) and +- 0.16 dex (Si) for all spectral types based on the comparison of the results obtained for stellar components of multiple systems. The derived abundances are consistent with the galactic evolution trends and observed chemical abundance distribution of earlier FGK-type stars in the solar neighbourhood. Besides, our analysis provides compatible abundances for stars in multiple systems. In addition, we studied the abundances of different galactic stellar populations. In this paper, we also explore the relation of the Mg and Si abundances of stars with and without known planets.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.20066-b31b1b.svg)](https://arxiv.org/abs/2407.20066) | **JOYS+: link between ice and gas of complex organic molecules. Comparing JWST and ALMA data of two low-mass protostars**  |
|| Y. Chen, et al. -- incl., <mark>H. Beuther</mark>, <mark>G. Perotti</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2024-07-30*|
|*Comments*| *42 pages (22 main text, 20 appendix); 27 figures (12 in main text, 15 in appendix); 5 tables (2 in main text, 3 in appendix) Accepted for publication in A&A*|
|**Abstract**|            A rich inventory of complex organic molecules (COMs) has been observed in high abundances in the gas phase toward Class 0 protostars. These molecules are suggested to be formed in ices and sublimate in the warm inner envelope close to the protostar. However, only the most abundant COM, methanol (CH3OH), has been firmly detected in ices before the era of James Webb Space Telescope (JWST). Now it is possible to detect the interstellar ices of other COMs and constrain their ice column densities quantitatively. We aim to determine the column densities of several oxygen-bearing COMs (O-COMs) in both gas and ice for two low-mass protostellar sources, NGC 1333 IRAS 2A and B1-c, as case studies in our JWST Observations of Young protoStars (JOYS+) program. By comparing the column density ratios w.r.t. CH3OH between both phases measured in the same sources, we can probe into the evolution of COMs from ice to gas in the early stages of star formation. We are able to fit the fingerprints range of COM ices between 6.8 and 8.8 um in the JWST/MIRI-MRS spectra of B1-c using similar components as recently used for IRAS 2A. We claim detection of CH4, OCN-, HCOO-, HCOOH, CH3CHO, C2H5OH, CH3OCH3, CH3OCHO, and CH3COCH3 in B1-c, and upper limits are estimated for SO2, CH3COOH, and CH3CN. The comparison of O-COM ratios w.r.t CH3OH between ice and gas shows two different cases. 1) the column density ratios of CH3OCHO and CH3OCH3 match well between the two phases, which may be attributed to a direct inheritance from ice to gas or strong chemical links with CH3OH. 2) the ice ratios of CH3CHO and C2H5OH w.r.t. CH3OH are higher than the gas ratios by 1-2 orders of magnitudes. This difference can be explained by the gas-phase reprocessing following sublimation, or different spatial distributions of COMs in the envelope.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.19839-b31b1b.svg)](https://arxiv.org/abs/2407.19839) | **Additive manufacturing applications in astronomy: a review**  |
|| Y. Chahid, et al. -- incl., <mark>W. Laun</mark> |
|*Appeared on*| *2024-07-30*|
|*Comments*| *27 pages, 17 figures, submitted to SPIE Astronomical Telescopes & Instrumentation, Advances in Optical and Mechanical Technologies for Telescopes and Instrumentation VI (Conference 13100, Paper 57)*|
|**Abstract**|            Despite the established role of additive manufacturing (AM) in aerospace and medical fields, its adoption in astronomy remains low. Encouraging AM integration in a risk-averse community necessitates documentation and dissemination of previous case studies. The objective of this study is to create the first review of AM in astronomy hardware, answering: where is AM currently being used in astronomy, what is the status of its adoption, and what challenges are preventing its widespread use? The review starts with an introduction to astronomical instruments size/cost challenges, alongside the role of manufacturing innovation. This is followed by highlighting the benefits/challenges of AM and used materials/processes in both space-based and ground-based applications. The review case studies include mirrors, optomechanical structures, compliant mechanisms, brackets and tooling applications that are either in research phase or are implemented.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.19152-b31b1b.svg)](https://arxiv.org/abs/2407.19152) | **IllustrisTNG in the HSC-SSP: No Shortage of Thin Disk Galaxies in TNG50**  |
|| D. Xu, et al. -- incl., <mark>J. Shi</mark> |
|*Appeared on*| *2024-07-30*|
|*Comments*| *13 pages, 7 figures, accepted for publication in ApJ*|
|**Abstract**|            We perform a thorough analysis of the projected shapes of nearby galaxies in both observations and cosmological simulations. We implement a forward-modeling approach to overcome the limitations in previous studies which hinder accurate comparisons between observations and simulations. We measure axis ratios of $z=0$ (snapshot 99) TNG50 galaxies from their synthetic Hyper Suprime-Cam Subaru Strategic Program (HSC-SSP) images and compare them with those obtained from real HSC-SSP images of a matched galaxy sample. Remarkably, the comparison shows excellent agreement between observations and the TNG50 simulation. Specifically, for galaxies with stellar masses $10\leq \log (M_{\star}/M_{\odot}) \leq 11.5$, we find $\lesssim 0.1\sigma$ tensions between the observation and the simulation, whereas previous studies found $\gtrsim 10\sigma$ tensions due to lack of thin galaxies in simulations. We reveal that low-mass galaxies ($M_{\star}\lesssim 10^{9.5}$) in TNG50 are thicker than their observed counterparts in HSC-SSP and attribute this to the spurious dynamical heating effects that artificially puff up galaxies. We also find that despite the overall broad agreement, TNG50 galaxies are more concentrated than the HSC-SSP ones at the low- and high-mass end of the stellar mass range of $9.0\leq \log (M_{\star}/M_{\odot}) \leq 11.2$ and are less concentrated at intermediate stellar masses. But we argue that the higher concentrations of the low-mass TNG50 galaxies are not likely the cause of their thicker/rounder appearances. Our study underscores the critical importance of conducting mock observations of simulations and applying consistent measurement methodologies to facilitate proper comparison with observations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.19388-b31b1b.svg)](https://arxiv.org/abs/2407.19388) | **A Phase-resolved View of "Heartbeat"-like variability in IGR J17091-3624 During the 2022 Outburst**  |
|| Q.-C. Shui, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-07-30*|
|*Comments*| *Accepted for publication in The Astrophysical Journal*|
|**Abstract**|            IGR J17091-3624, in addition to GRS 1915+105, is the only black hole X-ray binary that displays ``heartbeat"-like variability characterized by structured flares with high amplitudes. In this study, we conduct a detailed phase-resolved analysis of the recently identified ``heartbeat"-like Class X variability in IGR J17091-3624 during its 2022 outburst, utilizing data from NICER and NuSTAR observations. A shortage in the high-energy (>20 keV) X-ray flux is detected at peak phases of the soft X-ray flare at a ~15 sigma confidence level from the phase-folded light curves. Furthermore, our phase-resolved spectral analysis reveals variations in the spectral shape, particularly showing significant synchronous variations in the disk temperature and flux with the count rate. These findings imply that the flare is primarily driven by instabilities within the accretion disk, consistent with previous studies on the well-known Class rho variability in GRS 1915+105. However, we also observe a positive correlation between the disk temperature and flux over the flare cycle, which differs from a loop relation between the two parameters found in the Class rho variability. This could suggest differences in underlying physical processes between the two variability classes. Variations in the Componization component during flares are also observed: the electron temperature and covering fraction show anti-correlations with the disk flux, revealing potential interactions between the accretion disk and the corona during these flares.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.20118-b31b1b.svg)](https://arxiv.org/abs/2407.20118) | **Impact of Parameters in the Blazar Jet Magnetic Field Model on Axion-Like Particle Constraints**  |
|| L.-Q. Gao, X.-J. Bi, <mark>J. Li</mark>, P.-F. Yin |
|*Appeared on*| *2024-07-30*|
|*Comments*| *9 pages, 7 figures*|
|**Abstract**|            The interaction between axion-like particles (ALPs) and photons induces ALP-photon oscillations in astrophysical magnetic fields, leading to spectral distortions in the $\gamma$-ray spectrum of blazars. The primary uncertainty of this phenomenon may originate from the magnetic field within the jet of the blazar. While many studies have explored the effects of ALP-photon oscillations using typical values for jet magnetic field parameters, it is important to recognize that these parameters can be constrained by multi-wavelength observations. In this study, we utilize the high energy $\gamma$-ray spectrum of Mrk 421 obtained from MAGIC and Fermi-LAT observations. By employing multi-wavelength fitting with a one-zone synchrotron self-Compton model, we derive the parameters characterizing the magnetic field model within the jet, and investigate their impacts on the ALP constraints.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.20220-b31b1b.svg)](https://arxiv.org/abs/2407.20220) | **Ionospheric contributions to the excess power in high-redshift 21-cm power-spectrum observations with LOFAR**  |
|| S. Brackenhoff, et al. -- incl., <mark>S. Ghosh</mark> |
|*Appeared on*| *2024-07-30*|
|*Comments*| *22 pages, 20 figures. Accepted for publication in Monthly Notices of the Royal Astronomical Society (MNRAS)*|
|**Abstract**|            The turbulent ionosphere causes phase shifts to incoming radio waves on a broad range of temporal and spatial scales. When an interferometer is not sufficiently calibrated for the direction-dependent ionospheric effects, the time-varying phase shifts can cause the signal to decorrelate. The ionosphere's influence over various spatiotemporal scales introduces a baseline-dependent effect on the interferometric array. We study the impact of baseline-dependent decorrelation on high-redshift observations with the Low Frequency Array (LOFAR). Datasets with a range of ionospheric corruptions are simulated using a thin-screen ionosphere model, and calibrated using the state-of-the-art LOFAR Epoch of Reionisation pipeline. For the first time ever, we show the ionospheric impact on various stages of the calibration process including an analysis of the transfer of gain errors from longer to shorter baselines using realistic end-to-end simulations. We find that direction-dependent calibration for source subtraction leaves excess power of up to two orders of magnitude above the thermal noise at the largest spectral scales in the cylindrically averaged auto-power spectrum under normal ionospheric conditions. However, we demonstrate that this excess power can be removed through Gaussian process regression, leaving no excess power above the ten per cent level for a $5~$km diffractive scale. We conclude that ionospheric errors, in the absence of interactions with other aggravating effects, do not constitute a dominant component in the excess power observed in LOFAR Epoch of Reionisation observations of the North Celestial Pole. Future work should therefore focus on less spectrally smooth effects, such as beam modelling errors.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2407.19919/./figs/sigma_stat_sum1.png', 'tmp_2407.19919/./figs/sigma_stat_compl.png', 'tmp_2407.19919/./figs/gal_density_slice_z2neu.png', 'tmp_2407.19919/./figs/PCplotx_060.png', 'tmp_2407.19919/./figs/PCplotx_036.png', 'tmp_2407.19919/./figs/PCplotx_045.png']
copying  tmp_2407.19919/./figs/sigma_stat_sum1.png to _build/html/
copying  tmp_2407.19919/./figs/sigma_stat_compl.png to _build/html/
copying  tmp_2407.19919/./figs/gal_density_slice_z2neu.png to _build/html/
copying  tmp_2407.19919/./figs/PCplotx_060.png to _build/html/
copying  tmp_2407.19919/./figs/PCplotx_036.png to _build/html/
copying  tmp_2407.19919/./figs/PCplotx_045.png to _build/html/
exported in  _build/html/2407.19919.md
    + _build/html/tmp_2407.19919/./figs/sigma_stat_sum1.png
    + _build/html/tmp_2407.19919/./figs/sigma_stat_compl.png
    + _build/html/tmp_2407.19919/./figs/gal_density_slice_z2neu.png
    + _build/html/tmp_2407.19919/./figs/PCplotx_060.png
    + _build/html/tmp_2407.19919/./

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# $\Euclid$ preparation. Exploring the properties of proto-clusters in the Simulated   Euclid Wide Survey

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.19919-b31b1b.svg)](https://arxiv.org/abs/2407.19919)<mark>Appeared on: 2024-07-30</mark> -  _Submitted to Astronomy and Astrophysics, 24 pages, 28 figures_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>E. Bañados</mark>, <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** Galaxy proto-clusters are receiving an increased interest since most of the processes shaping the structure of clusters of galaxies and their galaxy population are happening at early stages of their formation.The Euclid Survey will provide a unique opportunity to discover a large number of proto-clusters over a large fraction of the sky ( 14 500 deg $^2$ ).In this paper, we explore the expected observational properties of proto-clusters in the Euclid Wide Survey by means of theoretical models and simulations.We provide an overview of the predicted proto-cluster extent, galaxy density profiles, mass-richness relations, abundance, and sky-filling as a function of redshift.Useful analytical approximations for the functions of these properties are provided.The focus is on the redshift range $z= 1.5$ to $4$ .We discuss in particular the density contrast with which proto-clusters can be observed against the background in the galaxy distribution if photometric galaxy redshifts are used as supplied by the ESA $\Euclid$ mission together with the ground-based photometric surveys.We show that the obtainable detection significance is sufficient to findlarge numbers of interesting proto-cluster candidates.For quantitative studies, additional spectroscopic follow-up is required to confirm the proto-clusters and establish their richness.

</div>

<div id="div_fig1">

<img src="tmp_2407.19919/./figs/sigma_stat_sum1.png" alt="Fig13.1" width="50%"/><img src="tmp_2407.19919/./figs/sigma_stat_compl.png" alt="Fig13.2" width="50%"/>

**Figure 13. -** ** Top:** Detection significance of proto-clusters as a function of aperture radius for three different redshift intervals.
The solid curves show the mean significance, while the dashed curves show the maxima.
A redshift interval of $\Delta_{z (50\%)}$ was used.
** Bottom:** Mean detection significance of proto-clusters as a function of the completeness limit, $\Delta{z (X\%)}$, in the range 30 to 90\% for three redshift intervals. The solid lines show the results for an aperture radius of 0.5 $r_{\rm pc}$ and dashed lines for 1 $r_{\rm pc}$.
 (*fig23*)

</div>
<div id="div_fig2">

<img src="tmp_2407.19919/./figs/gal_density_slice_z2neu.png" alt="Fig22" width="100%"/>

**Figure 22. -** Proto-clusters and their galaxies in a redshift slice around $z = 2$ with a redshift range according to the photometric redshift accuracy limits for 50\% completeness and $M_{\rm pc} \ge 10^{14}$\si{\solarmass}.
The dense core regions inside $r \le 0.5 r_{\rm pc}$ are shown by black circles and red dots for the galaxies, while the outer regions are shown in grey.
 (*fig32*)

</div>
<div id="div_fig3">

<img src="tmp_2407.19919/./figs/PCplotx_060.png" alt="Fig28.1" width="33%"/><img src="tmp_2407.19919/./figs/PCplotx_036.png" alt="Fig28.2" width="33%"/><img src="tmp_2407.19919/./figs/PCplotx_045.png" alt="Fig28.3" width="33%"/>

**Figure 28. -** 
Examples of three proto-clusters for the categories 3 to 5. The category and the redshift of the proto-clusters is given in the legend of the Figure.
 (*fig52*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.19919"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# The CARMENES search for exoplanets around M dwarfs$\fnmsep$$\thanks{Table \ref{tab:sample} is only available in electronic form at the CDS via anonymous ftp to cdsarc.u-strasbg.fr (130.79.128.5) or via http://cdsweb.u-strasbg.fr/cgi-bin/qcat?J/A+A/.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.19969-b31b1b.svg)](https://arxiv.org/abs/2407.19969)<mark>Appeared on: 2024-07-30</mark> -  _11 Pages, 5 figures. Accepted for publication in A&A_

</div>
<div id="authors">

H. M. Tabernero, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:**            We present the abundances of magnesium (Mg) and silicon (Si) for 314 dwarf stars with spectral types in the interval K7.0-M5.5 (Teff range ~4200-3050 K) observed with the CARMENES high-resolution spectrograph at the 3.5 m telescope at the Calar Alto Observatory. Our analysis employs the BT-Settl model atmospheres, the radiative transfer code Turbospectrum, and a state-of-the-art selection of atomic and molecular data. These Mg and Si abundances are critical for understanding both the chemical evolution and assembly of the Milky Way and the formation and composition of rocky planets. Our chemical abundances show a line-to-line scatter at the level of 0.1 dex for all studied spectral types. The typical error bar of our chemical abundance measurements is +- 0.11 dex (Mg) and +- 0.16 dex (Si) for all spectral types based on the comparison of the results obtained for stellar components of multiple systems. The derived abundances are consistent with the galactic evolution trends and observed chemical abundance distribution of earlier FGK-type stars in the solar neighbourhood. Besides, our analysis provides compatible abundances for stars in multiple systems. In addition, we studied the abundances of different galactic stellar populations. In this paper, we also explore the relation of the Mg and Si abundances of stars with and without known planets.         

</div>

<div id="div_fig1">

<img src="tmp_2407.19969/./MgFe.png" alt="Fig1.1" width="50%"/><img src="tmp_2407.19969/./SiFe.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** [X/Fe] vs. [Fe/H] for both Mg (top panel) and Si (bottom panel). Our target stars are represented by orange circles while the black open circles correspond to the abundances derived by for 1 111 FGK stars calculated by \citet{adi12}.  The typical error bar for each abundance is displayed in the bottom right of each panel. (*fig:abundances*)

</div>
<div id="div_fig2">

<img src="tmp_2407.19969/./Mg_multiple.png" alt="Fig2.1" width="50%"/><img src="tmp_2407.19969/./Si_multiple.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Comparison in both [Mg/H](top panel) and [Si/H](bottom panel) between the components in FGK$+$M (blue) and M$+$M (orange) systems. (*fig:binary*)

</div>
<div id="div_fig3">

<img src="tmp_2407.19969/./SiFevsfiron.png" alt="Fig4.1" width="50%"/><img src="tmp_2407.19969/./firon.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** Top panel: [Si/Fe] vs. $f_{\rm iron}$. The target stars are in orange while the black points correspond to the $f_{\rm iron}$ values computed from the \citet{adi12} abundances. The fit through the M dwarf values is denoted with a blue line. Bottom panel: Histogram for the $f_{\rm iron}$ values of our sample. (*fig:firon1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.19969"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\stkout}[1]{\ifmmode\text{\sout{\ensuremath{#1}}}\else\sout{#1}\fi}$
$\newcommand{\edited}[2]{\ifthenelse{\isempty{#1}}{\textbf{#2}}{\ifthenelse{\isempty{#2}}{\textcolor{gray}{\stkout{#1}}}{\textcolor{gray}{\stkout{#1}} \textbf{#2}}}}$
$\newcommand{\highlight}[1]{\uline{#1}}$
$\newcommand{\replacedecimal}[2]{\ifthenelse{\isin{.}{#1}}{\text{\StrBefore{#1}{.}}\ensuremath{\overset{#2}{.}}\text{\StrBehind{#1}{.}}}{#1\ensuremath{^{#2}}}}$
$\newcommand{\hms}[3]{\ensuremath{#1\overset{\text{h}}{\phantom{.}}#2\overset{\text{m}}{\phantom{.}}\replacedecimal{#3}{\text{s}}}}$
$\newcommand{\dms}[3]{\ensuremath{#1\overset{\circ}{\phantom{.}}#2\overset{\prime}{\phantom{.}}\replacedecimal{#3}{\prime\prime}}}$
$\newcommand{\arraystretch}{1.6}$</div>



<div id="title">

# JOYS+: link between ice and gas of complex organic molecules

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.20066-b31b1b.svg)](https://arxiv.org/abs/2407.20066)<mark>Appeared on: 2024-07-30</mark> -  _42 pages (22 main text, 20 appendix); 27 figures (12 in main text, 15 in appendix); 5 tables (2 in main text, 3 in appendix) Accepted for publication in A&A_

</div>
<div id="authors">

Y. Chen, et al. -- incl., <mark>H. Beuther</mark>, <mark>G. Perotti</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** A rich inventory of complex organic molecules (COMs) has been observed in high abundances in the gas phase toward Class 0 protostars. These molecules are suggested to be formed in ices and sublimate in the warm inner envelope close to the protostar. However, only the most abundant COM, methanol ( $\ce{CH3OH}$ ), has been firmly detected in ices before the era of _James Webb_ Space Telescope (JWST). Now it is possible to detect the interstellar ices of other COMs and constrain their ice column densities quantitatively. We aim to determine the column densities of several oxygen-bearing COMs (O-COMs) in both gas and ice for two low-mass protostellar sources, NGC 1333 IRAS 2A (hereafter IRAS 2A) and B1-c, as case studies in our JWST Observations of Young protoStars (JOYS+) program. By comparing the column density ratios with respect to $\ce{CH3OH}$ between both phases measured in the same sources, we can probe into the evolution of COMs from ice to gas in the early stages of star formation. The column densities of COMs in gas and ice are derived by fitting the spectra observed by the Atacama Large Millimeter/submillimeter Array (ALMA) and the JWST/Mid-InfraRed Instrument-Medium Resolution Spectroscopy (MIRI-MRS), respectively. The gas-phase emission lines are fit using local thermal equilibrium (LTE) models, and the ice absorption bands are fit by matching the infrared spectra measured in laboratories. The column density ratios of four O-COMs ( $\ce{CH3CHO}$ , $\ce{C2H5OH}$ , $\ce{CH3OCH3}$ , and $\ce{CH3OCHO}$ ) with respect to $\ce{CH3OH}$ are compared between ice and gas in IRAS 2A and B1-c. We are able to fit the fingerprints range of COM ices between 6.8 and 8.8 $\mu m$ in the JWST/MIRI-MRS spectra of B1-c using similar components as recently used for NGC 1333 IRAS 2A. We claim detection of $\ce{CH4}$ , $\ce{OCN^-}$ , $\ce{HCOO^-}$ , $\ce{HCOOH}$ , $\ce{CH3CHO}$ , $\ce{C2H5OH}$ , $\ce{CH3OCH3}$ , $\ce{CH3OCHO}$ , and $\ce{CH3COCH3}$ in B1-c, and upper limits are estimated for $\ce{SO2}$ , $\ce{CH3COOH}$ , and $\ce{CH3CN}$ . The total abundance of O-COM ices is constrained to be 15 \% with respect to $\ce{H2O}$ ice, 80 \% of which is dominated by $\ce{CH3OH}$ . The comparison of O-COM ratios with respect to $\ce{CH3OH}$ between ice and gas shows two different cases. On one hand, the column density ratios of $\ce{CH3OCHO}$ and $\ce{CH3OCH3}$ match well between the two phases, which may be attributed to a direct inheritance from ice to gas or strong chemical links with $\ce{CH3OH}$ . On the other hand, the ice ratios of $\ce{CH3CHO}$ and $\ce{C2H5OH}$ with respect to $\ce{CH3OH}$ are higher than the gas ratios by 1--2 orders of magnitudes. This difference can be explained by the gas-phase reprocessing following sublimation, or different spatial distributions of COMs in the envelope, which is an observational effect since ALMA and JWST are tracing different components in a protostellar system. The firm detection of COM ices other than $\ce{CH3OH}$ is reported in another well-studied low-mass protostar, B1-c, following the recent detection in NGC 1333 IRAS 2A. The column density ratios of four O-COMs with respect to $\ce{CH3OH}$ show both similarities and differences between gas and ice. Although the straightforward explanations would be the direct inheritance from ice to gas and the gas-phase reprocessing, respectively, other possibilities such as different spatial distributions of molecules cannot be excluded.

</div>

<div id="div_fig1">

<img src="tmp_2407.20066/./plots/obs_vs_lab_mix_2col.png" alt="Fig13" width="100%"/>

**Figure 13. -** Comparison between the JWST/MIRI spectrum of B1-c (gray) and the lab spectra (colors) in the COMs fingerprints range of 6.8--8.8 $\mu$m. Each panel focuses on one species and shows the comparison between the observed B1-c spectrum and the lab spectra with different mixing constituents; except for panel (c) which shows the lab spectra of \ce{HCOO^-} under different temperatures. The observed spectrum along with the 3$\sigma$ level is shown in light gray; except for panel (a), which blows up the \ce{CH4} band at $\sim$7.7 $\mu$ and the observed spectrum is plotted in black for clarity. In each panel, the spectrum in blue corresponds to the pure ice, and the best-fit spectrum to the observations is highlighted with a thicker red line. Similar comparison but for lab spectra under different temperatures is shown in Figs. \ref{fig:obs_vs_lab_T_1} and \ref{fig:obs_vs_lab_T_2}. (*fig:obs_vs_lab_mix*)

</div>
<div id="div_fig2">

<img src="tmp_2407.20066/./plots/steps_summary_v3.png" alt="Fig1" width="100%"/>

**Figure 1. -** Panels (a)--(d) show the four steps to isolate the fingerprints range of COM ices between 6.8 and 8.8 $\mu$m from the original JWST/MIRI-MRS spectrum of B1-c: (a) fit a global continuum and convert the spectrum to optical depth scale; (b) subtract silicate features at $\sim$9.8 and 18 $\mu$m; (c) trace a local continuum in the 6.8--8.8 $\mu$m range to isolate the weak bands of COM ices from other strong features; (d) trace a baseline of the gas-phase lines in absorption to restore the band profiles of ices. Panel (e) shows the isolation of the \ce{CH3OH} ice band at 9.74 $\mu$m by tracing a local continuum. Orange shaded regions in panels (a) and (e) show the selected wavelength ranges for the polynomial fitting. (*fig:fitting_steps*)

</div>
<div id="div_fig3">

<img src="tmp_2407.20066/./plots/obs_vs_lab_T_1.png" alt="Fig14" width="100%"/>

**Figure 14. -** Same as Fig. \ref{fig:obs_vs_lab_mix} but for comparison among different temperatures. Here we take two species, \ce{CH3OCHO} and \ce{C2H5OH}, as examples; the remaining species (\ce{CH3CHO}, \ce{CH3OCH3}, and \ce{CH3COCH3}) are shown in Fig. \ref{fig:obs_vs_lab_T_2}. The left and right columns show the lab spectra of pure ices and ice mixtures, respectively. In the pure-ice panels, the spectra with crystalline features are highlighted in thicker red lines. The corresponding temperatures indicate the upper limit of crystallines. In the mixed-ice panels, the spectra with the lowest temperature (15 K) are highlighted in thicker blue lines; they are also the spectra used in the overall fitting. (*fig:obs_vs_lab_T_1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.20066"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

149  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
